# Gene expression & surface protein TIL 

## Divide adata into: Protein and RNA 

In [ ]:
# Libraries
import anndata as ad
import matplotlib as plt
import numpy as np
import pandas as pd
import sc_atlas_helpers as ah
import scanpy as sc
from matplotlib.pyplot import rc_context
from scipy.stats import median_abs_deviation

In [ ]:
from functools import partial

import altair as alt

In [ ]:
import seaborn as sns

In [ ]:
import decoupler as dc

In [ ]:
alt.renderers.enable("png")
alt.data_transformers.disable_max_rows()

In [ ]:
resDir = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/"
inputDir = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/"

In [ ]:
adata = sc.read_h5ad(f"{resDir}adata_merged_til_all_genes.h5ad")

In [ ]:
adata

In [ ]:
adata.obs

In [ ]:
adata.var[adata.var["gene_ids"]=="ENSMUSG00000060594"]

In [ ]:
adata.var[adata.var["gene_ids"]=="ENSMUSG00000023078"]

In [ ]:
adata.obs_names = (
    adata.obs_names + "_" + adata.obs["sample_id"].astype(str)
)  # to avoid duplicate obs names i combine obs names + sample_id

In [ ]:
adata.obs

## Protein

In [ ]:
protein = adata[:, adata.var["feature_types"] == "Antibody Capture"].copy()

In [ ]:
#protein.obs_names = (
#    protein.obs_names + "_" + protein.obs["sample_id"].astype(str)
#)  # to avoid duplicate obs names i combine obs names + sample_id

In [ ]:
protein.var

In [ ]:
protein.layers["counts"] = protein.X.copy()

In [ ]:
sc.pp.normalize_total(protein)

In [ ]:
sc.pp.log1p(protein)

In [ ]:
sc.pp.pca(protein)

In [ ]:
sc.pp.neighbors(protein)  # why can't we just work with the default neighbors?

In [ ]:
sc.tl.leiden(protein, key_added="protein_leiden", n_iterations=2)

In [ ]:
sc.tl.umap(protein)

In [ ]:
protein.obsp["protein_connectivities"] = protein.obsp["connectivities"].copy()

In [ ]:
protein

### Protein - Visualization

In [ ]:
# sc.tl.umap(protein)
# sc.pl.umap(protein, color="protein_leiden", size=10)

In [ ]:
# sc.pl.umap(protein, color=["PD1_TotalSeqC","CD69_TotalSeqC","CD44_TotalSeqC"], size=10, vmax="p99", vmin=0, cmap="Reds" )

In [ ]:
# sc.pl.umap(protein, color=["ICOS_TotalSeqC","CD103_TotalSeqC"], size=10, vmax="p99", vmin=0, cmap="Reds" )

In [ ]:
# sc.pl.umap(protein, color="condition", size=10, groups = "11mix")

### RNA

In [ ]:
rna = adata[:, adata.var["feature_types"] == "Gene Expression"].copy()

In [ ]:
#rna_bottom = adata_bottom[:, adata_bottom.var["feature_types"] == "Gene Expression"].copy()

In [ ]:
#rna.obs_names = (
#    rna.obs_names + "_" + rna.obs["sample_id"].astype(str)
#)  # to avoid duplicate obs names i combine obs names + sample_id


In [ ]:
rna.layers["counts"] = rna.X.copy()

In [ ]:
rna.var[rna.var["gene_ids"]=="ENSMUSG00000060594"]

In [ ]:
rna.var["mito"] = rna.var_names.str.startswith("mt")
rna.var["ribo"] = rna.var_names.str.startswith("rb")
rna.var["hb"] = rna.var_names.str.startswith("hb")
sc.pp.calculate_qc_metrics(
    rna,
    qc_vars=["mito", "ribo", "hb"],
    inplace=True,
    percent_top=[20],
    log1p=True,
)

In [ ]:
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)

In [ ]:
#sc.pp.highly_variable_genes(rna, n_top_genes=2000, batch_key="sample_id")

In [ ]:
sc.pp.highly_variable_genes(rna, n_top_genes=1000, batch_key="sample_id")

In [ ]:
sc.pp.pca(rna)
sc.pp.neighbors(rna, n_neighbors=30)
sc.tl.umap(rna)
sc.tl.leiden(rna, key_added="rna_leiden")

In [ ]:
sc.tl.leiden(rna, resolution=0.5, key_added="rna_leiden_05")

In [ ]:
rna

In [ ]:
sc.pp.filter_genes(rna, min_cells=10)

In [ ]:
rna.var[rna.var["gene_ids"]=="ENSMUSG00000060594"]

In [ ]:
rna

In [ ]:
# ah.pl.umap_qc_metrics(rna, vmax_total_counts=20000, vmax_n_genes=2000)
sc.pl.umap(
    rna,
    color=["pct_counts_mito", "total_counts", "n_genes_by_counts"],
    cmap="inferno",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False,
)

In [ ]:
sc.pl.umap(rna, color=["rna_leiden_05", "rna_leiden"], size=10, legend_loc="on data")

In [ ]:
with rc_context({"figure.figsize": (4.5, 3)}):
    sc.pl.violin(
        rna,
        ["n_genes_by_counts", "pct_counts_mito"],
        groupby="rna_leiden_05",
        stripplot=False,  # remove the internal dots
        inner="box",  # adds a boxplot inside violins
    )

In [ ]:
## sp
rna.obs["is_outlier_counts"] = ah.pp.is_outlier(
    rna, "log1p_total_counts", n_mads=1.5, groupby="sample_id"
)
rna.obs["is_outlier_genes"] = ah.pp.is_outlier(
    rna, "log1p_n_genes_by_counts", n_mads=1.5, groupby="sample_id"
)
rna.obs["is_outlier_top_20"] = ah.pp.is_outlier(
    rna, "pct_counts_in_top_20_genes", n_mads=1.5, groupby="sample_id"
)
rna.obs["is_outlier_mito"] = ah.pp.is_outlier(
    rna, "pct_counts_mito", n_mads=3, groupby="sample_id"
)

rna.obs["is_outlier"] = (
    np.sum(
        rna.obs.loc[
            :,
            [
                "is_outlier_counts",
                "is_outlier_genes",
                "is_outlier_top_20",
                "is_outlier_mito",
            ],
        ],
        axis=1,
    )
    >= 2
)

In [ ]:
rna_filtered = rna[~rna.obs["is_outlier"]].copy()

In [ ]:
rna_filtered

In [ ]:
sc.pl.umap(
    rna_filtered,
    color=["pct_counts_mito", "total_counts", "n_genes_by_counts"],
    cmap="inferno",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False,
)

In [ ]:
sc.pl.umap(
    rna_filtered, color=["rna_leiden_05", "rna_leiden"], size=10, legend_loc="on data"
)

In [ ]:
with rc_context({"figure.figsize": (4.5, 3)}):
    sc.pl.violin(
        rna_filtered,
        ["n_genes_by_counts", "pct_counts_mito"],
        groupby="rna_leiden_05",
        stripplot=False,  # remove the internal dots
        inner="box",  # adds a boxplot inside violins
    )

### Visualization RNA & Protein

In [ ]:
#### Visualization function
def embedding_chart(df: pd.DataFrame, coord_pat: str, *, size=5) -> alt.Chart:
    """Make schema for coordinates, like sc.pl.embedding."""
    x, y = df.columns[df.columns.str.contains(coord_pat)]
    return (
        alt.Chart(plotdf, height=300, width=300)
        .mark_circle(size=size)
        .encode(
            x=alt.X(x, axis=None),
            y=alt.Y(y, axis=None),
        )
    )


def umap_chart(df: pd.DataFrame, **kwargs) -> alt.Chart:
    """Like sc.pl.umap, but just the coordinates."""
    return embedding_chart(df, "umap", **kwargs)


def encode_color(
    c: alt.Chart, col: str, *, qdomain=(0, 1), scheme: str = "lightgreyred"
) -> alt.Chart:
    """Add colors to an embedding plot schema."""
    base = c.properties(title=col)
    if pd.api.types.is_categorical_dtype(c.data[col]):
        return base.encode(color=col)
    else:
        return base.encode(
            color=alt.Color(
                col,
                scale=alt.Scale(
                    scheme=scheme,
                    clamp=True,
                    domain=list(c.data[col].quantile(qdomain)),
                    nice=True,
                ),
            )
        )

In [ ]:
protein.obs

In [ ]:
rna.obs

In [ ]:
rna.obsm["protein"] = protein.to_df()
rna.obsm["protein_umap"] = protein.obsm["X_umap"]
rna.obs["protein_leiden"] = protein.obs["protein_leiden"]
rna.obsp["rna_connectivities"] = rna.obsp["connectivities"].copy()
rna.obsp["protein_connectivities"] = protein.obsp["protein_connectivities"]

In [ ]:
sc.tl.umap(rna)

In [ ]:
sc.pl.umap(rna, color=["rna_leiden", "protein_leiden"], size=10)
sc.pl.embedding(rna, basis="protein_umap", color=["rna_leiden", "protein_leiden"], size=10)

## Both modalities together

### Visualizing

In [ ]:
# Plotting protein on rna
plotdf = sc.get.obs_df(
    rna,
    obsm_keys=[("X_umap", i) for i in range(2)]
    + [("protein", i) for i in rna.obsm["protein"].columns],
)
(
    alt.concat(
        *map(
            partial(encode_color, umap_chart(plotdf), qdomain=(0, 0.95)),
            plotdf.columns[2:10],
        ),
        columns=2
    )
    .resolve_scale(color="independent")
    .configure_axis(grid=False)
)

In [ ]:
rna

### Clustering

In [ ]:
def join_graphs_max(g1: "sparse.spmatrix", g2: "sparse.spmatrix"):
    """Take the maximum edge value from each graph."""
    out = g1.copy()
    mask = g1 < g2
    out[mask] = g2[mask]

    return out

In [ ]:
rna.obsp["connectivities"] = join_graphs_max(
    rna.obsp["rna_connectivities"], rna.obsp["protein_connectivities"]
)

In [ ]:
sc.tl.leiden(rna, key_added="joint_leiden")

In [ ]:
sc.tl.leiden(rna, key_added="joint_leiden_15", resolution=1.5)

In [ ]:
sc.pl.umap(rna, color=["joint_leiden","joint_leiden_15"], size=5)

### Gather data

In [ ]:
adata

In [ ]:
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
adata

In [ ]:
rna

In [ ]:
rna.X

In [ ]:
adata.X[:,(adata.var["feature_types"]=="Gene Expression").values]

In [ ]:
adata.X[:,(adata.var["feature_types"]=="Gene Expression").values] = rna.X

In [ ]:
adata.X[:,(adata.var["feature_types"]=="Antibody Capture").values] = protein.X

In [ ]:
adata.X[:,(adata.var["feature_types"]=="Antibody Capture").values] = protein.X

In [ ]:
adata.obs

In [ ]:
rna.obs

In [ ]:
adata.obsm.update(rna.obsm)

In [ ]:
adata.obs[rna.obs.columns] = rna.obs

In [ ]:
adata

### Labelling

In [ ]:
sc.pl.umap(adata, color= ["joint_leiden","sample_id"])

In [ ]:
adata

In [ ]:
## sp
adata.obs["is_outlier_counts"] = ah.pp.is_outlier(
    adata, "log1p_total_counts", n_mads=1.5, groupby="sample_id"
)
adata.obs["is_outlier_genes"] = ah.pp.is_outlier(
    adata, "log1p_n_genes_by_counts", n_mads=1.5, groupby="sample_id"
)
adata.obs["is_outlier_top_20"] = ah.pp.is_outlier(
    adata, "pct_counts_in_top_20_genes", n_mads=1.5, groupby="sample_id"
)
adata.obs["is_outlier_mito"] = ah.pp.is_outlier(
    adata, "pct_counts_mito", n_mads=3, groupby="sample_id"
)

adata.obs["is_outlier"] = (
    np.sum(
        rna.obs.loc[
            :,
            [
                "is_outlier_counts",
                "is_outlier_genes",
                "is_outlier_top_20",
                "is_outlier_mito",
            ],
        ],
        axis=1,
    )
    >= 2
)

In [ ]:
adata_filtered = adata[~adata.obs["is_outlier"]].copy()

In [ ]:
adata

In [ ]:
adata_filtered

In [ ]:
adata = adata_filtered

### Subset adata bottom part check markers and integrate back

In [ ]:
sc.pl.umap(adata, color= ["joint_leiden","sample_id"])

## Cell type annotation from marker genes

### Marker genes

* Tissue Resident Memory T (TRM) 
CD103 
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6104123/

- ICOS

 https://pubmed.ncbi.nlm.nih.gov/34932944/

 Cycling CD8+ T 
-  Mki67 +

-  T cell subset
-  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7115982/

In [ ]:
# Markers from publication doi: 10.1038/s41568-019-0235-4
#CD8+ T cell states in human cancer: insights from single-cell analysis
# Reference study: Zhang (CRC) Human 
naive_markers = ["Lef1","Sell","Ccr7","Tcf7","Cd27","Cd28","S1pr1"] # Naive-like
effector_memory_markers = ["Gzmk","Cxcr4","Cxcr3","Cd44"] #Predysfunctional
exhausted_markers = ["Havcr2","Pdcd1","Ifng","Itgae"] #  Dysfunctional #Layn, Cxcl13 not found
activated_effector_memory_marker  = ["Cx3cr1","Klrg1","Prf1","Tbx21","Eomes","S1pr1","S1pr5"] # Temra, #Cytotoxic #not found "Fcgr3a","Fgfbp2""Gzmh"

#* not described in table 1 
central_memory_markers = ["Gpr183","Ccr7","Sell","Il7r","Cd27","Cd28","Gzma","Ccl5","S1pr1","Gzma"] # not described in table 1 
tissue_resident_memory_markers = ["Cd6","Myadm","Capg","Rora","Nr4a1","Nr4a2","Nr4a3","Cd69","Itgae"] # not described in table 1  #not found "Cxcl1" ,"Xcl2"

#**Tumour type-specific T cell subsets that are not included in Table 1
intraepitheliel_lymphocytes_markers = ["Cd160","Klrc1","Klrc2","Klrc3","Ikzf2","Entpd1","Cd69","Itgae","Nr4a1","Nr4a2"] #IEL #"Kir2dl4","Timigd2"
mucosal_associated_invariant_markers = ["Slc4a10","Zbtb16","Klrb1","Rora"] # MAIT # not found "Rorc"

In [ ]:
# Plotting protein on rna
plotdf = sc.get.obs_df(
    rna,
    obsm_keys=[("X_umap", i) for i in range(2)]
    + [("protein", i) for i in rna.obsm["protein"].columns],
)
(
    alt.concat(
        *map(
            partial(encode_color, umap_chart(plotdf), qdomain=(0, 0.95)),
            plotdf.columns[2:10],
        ),
        columns=2
    )
    .resolve_scale(color="independent")
    .configure_axis(grid=False)
)

In [ ]:
sc.pl.umap(adata, color=exhausted_markers,
    cmap="Reds",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False)

In [ ]:
ax = sc.pl.dotplot(adata, exhausted_markers, groupby='joint_leiden', log=True)

In [ ]:
#Early disfunciton markers https://www.nature.com/articles/s41577-021-00574-3/figures/2 
sc.pl.umap(adata, color=["Lag3","Cd38","Entpd1","Havcr2","Tox","Gzmk"],
    cmap="Reds",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False)

In [ ]:
ax = sc.pl.dotplot(adata, ["Lag3","Cd38","Entpd1","Havcr2","Tox","Gzmk"], groupby='joint_leiden', log=True)

In [ ]:
sc.pl.umap(adata, color=central_memory_markers,
    cmap="Reds",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False)

In [ ]:
ax = sc.pl.dotplot(adata, central_memory_markers, groupby='joint_leiden', log=True)

In [ ]:
#tissue_resident_memory_markers
mks =  ["Cd3e","Itgae","Cd8a","Zfp683","Ifng","Gzma","Gzmb","Prf1","Pdcd1","Ctla4","Havcr2","Lag3"] #https://www.nature.com/articles/s41416-023-02202-4/figures/2
sc.pl.umap(adata, color=mks,
    cmap="Reds",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False)

In [ ]:
ax = sc.pl.dotplot(adata, ["Cd3e","Itgae","Cd8a","Zfp683","Ifng","Gzma","Gzmb","Prf1","Pdcd1","Ctla4","Havcr2","Lag3"], groupby='joint_leiden', log=True)

### Annotate

In [ ]:
sc.pl.umap(adata, color='joint_leiden')

In [ ]:
annotation_dict = {
    '0':'Exhasuted/Early to late dysfunctional',
    '1':'Exhasuted/Early to late dysfunctional',
    '2':'Exhasuted/Early to late dysfunctional',
    '3':'Trm/Tcm/Naive/CTL',
    '4':'Trm/Tcm/Naive/CTL',
    '5':'Exhasuted/Early to late dysfunctional',
    '6':'Exhasuted/Early to late dysfunctional'}

In [ ]:
## Add cell type column based on annotation
adata.obs['cell_type'] = [annotation_dict[clust] for clust in adata.obs['joint_leiden']]

# Visualize
sc.pl.umap(adata, color='cell_type')

In [ ]:
#adata.write_h5ad(f"{resDir}/adata_merged_til_annotated.h5ad")

In [ ]:
adata.write_h5ad(f"{resDir}/adata_merged_til_annotated_new.h5ad")